In [0]:
from pyspark.sql.functions import *
from pyspark.sql.window import Window

In [0]:
class Transformer:
    def __init__(self):
        pass

    def transform(self,inputDfs):
        pass

class AirpodsAfterIphoneTransformer(Transformer):
    def transform(self,inputDfs):
        """
         Transformation 1 : find the customers who brought airpods after buying iPhone

        """
        transactionInputDf=inputDfs.get('transactionInputDf')
        print('transactionInputDf in transform' )
        transactionInputDf.display()
        windowSpec=Window.partitionBy('customer_id').orderBy('transaction_date')
        transformDf=transactionInputDf.withColumn('next_product_name',lead('product_name').over(windowSpec))
        print('Airpods after buying iPhone')
        transformDf.orderBy('customer_id','transaction_date','product_name').display()

        filterDf = transformDf.filter(
            (col('product_name') == 'iPhone') & 
         (col('next_product_name') == 'AirPods')
            )

        filterDf.orderBy('customer_id','transaction_date','product_name').display()

        customerInputDf=inputDfs.get('customerInputDf')
        customerInputDf.display()
        # joinDF=customerInputDf.join(customerInputDf,'customer_id')
        joinDF=customerInputDf.join(
            broadcast(filterDf),
            'customer_id')

        print('joined DF')
        joinDF.display()
        return joinDF.select(
            'customer_id',
            'customer_name',
            'location'
        )


class OnlyAirpodsAndIphone(Transformer):
    def transform(self,inputDfs):
        '''
        customers who brought only Airpods and iPhone only!
        '''
        transactionInputDf=inputDfs.get('transactionInputDf')

        groupedDf=transactionInputDf.groupBy('customer_id').agg(
            collect_set('product_name').alias('product_set')
        )
        groupedDf.display()
        


        filterDf = groupedDf.filter(
           (array_contains(col('product_set'),'AirPods')) &
           (array_contains(col('product_set'),'iPhone'))&
            (size(col('product_set'))==2)
                 
            )

        filterDf.display()

        customerInputDf=inputDfs.get('customerInputDf')
        customerInputDf.display()
        joinDF=customerInputDf.join(
            broadcast(filterDf),
            'customer_id')

        print('joined DF,customers who brought Airpods and iPhone only')
        joinDF.display()
        return joinDF.select(
            'customer_id',
            'customer_name',
            'location'
        )

